# 1. 损失函数
## L1 loss
X:[1,2,3]   <br>
Y:[1,2,5]   <br>
L1 loss: (0+0+2)/3 = 0.6 （mean）
## MSE
X:[1,2,3]   <br>
Y:[1,2,5]   <br>
(0+0+2^2)/3 = 1.3 
## CrossEntropy
-log(e^0.2 / (e^0.1 + e^0.2 + e^0.3))
![](/storage/pt/AW_STUDY/pytorch-tutorial-tudui/study_self/md_img/CE.png)


In [2]:
import torch
from torch import nn
from torch.nn import L1Loss

inputs = torch.tensor([1,2,3],dtype=torch.float32)
targets = torch.tensor([1,2,5],dtype=torch.float32)

inputs = torch.reshape(inputs,(1,1,1,3)) # batch_size = 1.channel =1,1行3列
targets = torch.reshape(targets,(1,1,1,3))

loss_l1 = L1Loss(reduction='mean') # 可加入 reduction 设计计算方式,eg:reduction='sum' 输出为2, 'mean'输出为2/3
result = loss_l1(inputs,targets)

loss_mse = nn.MSELoss()
result_mse = loss_mse(inputs,targets)

print(result)
print(result_mse)

tensor(0.6667)
tensor(1.3333)


In [4]:
import torch
from torch import nn

# CE
x = torch.tensor([0.1, 0.2, 0.3]).reshape(1,3)
y = torch.tensor([1])

loss_ce = nn.CrossEntropyLoss()

result_ce = loss_ce(x,y)
print(result_ce)


tensor(1.1019)


# 2. Backwards

In [9]:
import torchvision
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Linear
from torch.nn.modules.flatten import Flatten
from torch.utils.data import DataLoader

dataset = torchvision.datasets.CIFAR10('../dataset', train=False,
                                       transform=torchvision.transforms.ToTensor(),
                                       download=True)

dataloader = DataLoader(dataset,batch_size=1)


class Tudui(nn.Module):
    # 初始化
    def __init__(self):
        # 继承父类 初始化
        super(Tudui, self).__init__()
        # 引入 Seqential
        self.model1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x

# 计算交叉熵
loss = nn.CrossEntropyLoss()


tudui = Tudui()
cnt = 0
for data in dataloader:
    imgs, targets = data
    print(f"imgs shape:{imgs.shape}, tragerts shape:{targets.shape}")
    outputs = tudui(imgs)
    # print(f"outputs.shape:{outputs.shape}")

    result_loss = loss(outputs, targets)
    # print(f"outputs:{outputs}")
    print(f"targets:{targets}")
    print(f"{cnt}. result_loss:{result_loss}")

    result_loss.backward()  # 计算出来的 loss 值有 backward 方法属性，反向传播来计算每个节点的更新
    cnt+=1
    
    '''
    输出
    tensor([[-0.0549, -0.0800, -0.0625,  0.0550,  0.0861,  0.0304, -0.1175,  0.0740,
          0.0017,  0.0224]], grad_fn=<AddmmBackward>)
    tensor([9])
    tensor([[-0.0460, -0.0763, -0.0580,  0.0765,  0.0697,  0.0213, -0.1197,  0.0781,
          0.0140,  0.0327]], grad_fn=<AddmmBackward>)
    tensor([9])
    tensor([[-0.0372, -0.0863, -0.0581,  0.0738,  0.0784,  0.0194, -0.1176,  0.0831,
          0.0115,  0.0307]], grad_fn=<AddmmBackward>)
    ...
    '''   


Files already downloaded and verified
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([3])
0. result_loss:2.2714781761169434
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([8])
1. result_loss:2.151431083679199
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([8])
2. result_loss:2.1485912799835205
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([0])
3. result_loss:2.3459925651550293
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([6])
4. result_loss:2.3558437824249268
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([6])
5. result_loss:2.369126558303833
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([1])
6. result_loss:2.359736442565918
imgs shape:torch.Size([1, 3, 32, 32]), tragerts shape:torch.Size([1])
targets:tensor([6])
7. resu

# 3. 优化器
* 优化器根据梯度对参数进行调整，降低损失
* torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False, , maximize=False, foreach=None)
    * 参数params 代表网络模型中的参数,必须项目
    * 参数lr = learining rate，学习速率。
* lr太大会造成模型训练起来不稳定，太小训练比较慢。建议刚开始lr大一些，后面就小一些。其余参数为算法SGD本身特有的，初学时可以直接使用默认即可。

    ## 操作步骤:
1. 对每个节点对应的梯度清零。 optim.zero_grad()
2. 反向传播，计算节点梯度。result_loss.backward()
3. 根据节点中的梯度对参数进行调优。optim.step()

In [1]:
import torch
import torchvision
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Linear
from torch.nn.modules.flatten import Flatten
from torch.utils.data import DataLoader

dataset = torchvision.datasets.CIFAR10('../dataset', train=False,
                                       transform=torchvision.transforms.ToTensor(),
                                       download=True)

dataloader = DataLoader(dataset,batch_size=1)


class Tudui(nn.Module):
    # 初始化
    def __init__(self):
        # 继承父类 初始化
        super(Tudui, self).__init__()
        # 引入 Seqential
        self.model1 = Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x

# 计算交叉熵
loss = nn.CrossEntropyLoss()
tudui = Tudui()
# 加入优化器
optim = torch.optim.SGD(tudui.parameters(),lr=0.01)  # SGD 随机梯度下降

for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = tudui(imgs)
        result_loss = loss(outputs,targets)
        # 将网络中每一个梯度可调节参数调为0
        optim.zero_grad()
        # 调入损失函数的反向传播(每一参数节点的梯度)
        result_loss.backward()
        # 对模型参数进行调优
        optim.step()
        running_loss = running_loss + result_loss
    print(running_loss)

/home/kunyue/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
tensor(18769.3047, grad_fn=<AddBackward0>)
tensor(16153.7959, grad_fn=<AddBackward0>)
tensor(15344.2207, grad_fn=<AddBackward0>)
tensor(15876.4121, grad_fn=<AddBackward0>)
tensor(17692.9277, grad_fn=<AddBackward0>)
tensor(20405.1445, grad_fn=<AddBackward0>)
tensor(22024.5215, grad_fn=<AddBackward0>)
tensor(23740.4551, grad_fn=<AddBackward0>)
tensor(24599.4336, grad_fn=<AddBackward0>)
tensor(24814.7227, grad_fn=<AddBackward0>)
tensor(25073.4219, grad_fn=<AddBackward0>)
tensor(25709.6719, grad_fn=<AddBackward0>)
tensor(26719.3125, grad_fn=<AddBackward0>)
tensor(26720.6992, grad_fn=<AddBackward0>)
tensor(27779.2520, grad_fn=<AddBackward0>)
tensor(29035.8906, grad_fn=<AddBackward0>)
tensor(31949.9902, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)


# 4. 现有模型网络的使用与更改
以VGG16网络模型为例，最后分类是1000类，而使用的CIFAR10数据集需要最后分成10类，因此需要进行网络模型的修改。
* 直接添加线性层
* 修改最后线性层的参数

In [10]:
import torchvision

# trauin_data = torchvision.datasets.ImageNet("../dataset",split="train",download=True,transform=torchvision.transforms.ToTensor())   # 这个数据集没有办法再公开的访问了    
vgg16_true = torchvision.models.vgg16(pretrained=True) # 下载卷积层对应的参数是多少、池化层对应的参数时多少，这些参数时ImageNet训练好了的
vgg16_false = torchvision.models.vgg16(pretrained=False) # 没有预训练的参数
print("ok")
print(vgg16_true)

# 如何改进现有的网络去实现自己的目标
# Vgg16训练好的模型，最后为1000类，而CIFAR10为10类
train_data = torchvision.datasets.CIFAR10("../dataset", train=True, download=True,
                                          transform=torchvision.transforms.ToTensor())

## 第一种实现，在 最后 添加Linear层，将1000类转换成10类
# vgg16_true.add_module("add_linear", nn.Linear(1000, 10))

## 第二种实现，在 最后 添加Linear层，将1000类转换成10类
# vgg16_true.classifier.add_module("7", nn.Linear(1000, 10))

# 第三种，直接将最后一层线性层输出改成10类, 即直接修改VGG16模型最后Linear层的参数 把4096,1000改成4096,10
vgg16_true.classifier[6] = nn.Linear(4096, 10)  # 标明[6] 更改的就是6

print(vgg16_true)

/home/kunyue/anaconda3/envs/yolov5_frog38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kunyue/anaconda3/envs/yolov5_frog38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/kunyue/anaconda3/envs/yolov5_frog38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ok
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

# 5. 网络模型的保存与读取

In [17]:
import torch
import torchvision
from torch import nn

# # 加载方式1->保存方式1,直接加载权重
# model_dict = torch.load("/storage/pt/AW_STUDY/pytorch-tutorial-tudui/weight/vgg16-397923af.pth")
# print(model_dict)

# 加载方式2->保存方式2,加载模型字典参数
vgg16 = torchvision.models.vgg16(pretrained=False)
vgg16.load_state_dict(torch.load("/storage/pt/AW_STUDY/pytorch-tutorial-tudui/weight/vgg16-397923af.pth")) # 显示模型架构
print(vgg16) # 打印架构

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch
import torchvision
from torch import nn

# 陷阱1： 在model_save.py中保存了模型，在当前代码中加载模型，由于当前文件缺少模型结构定义所以报错
# 一般在抬头引入 from model_save import * 引入所有定义则不会报错
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.conv1 = nn.Conv2d(3, 64,kernel_size=3)

    def forward(self, x):
        x = self.conv1(x)
        return x

tudui = Tudui()
torch.save(tudui, "../weight/tudui.pth")

In [20]:
model3 = torch.load("../weight/tudui.pth")
print(model3)

Tudui(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
)
